# Create predcitors with Prophet and DeepAR+

* **About 40 mins may be elapsed**

In [1]:
import boto3
from time import sleep
import pandas as pd
import json
import time
import pprint
import numpy as np

In [2]:
# Recover variables 
%store -r

In [3]:
session = boto3.Session(region_name=region)
forecast = session.client(service_name='forecast')

## Hyperparameters

In [4]:
forecastHorizon = 31 # the number of forecasting
NumberOfBacktestWindows = 2 # the number of Backtests
BackTestWindowOffset = 31 # the point from the end of the dataset where the data is split for model training and testing 
ForecastFrequency = "D"

Detailed parameter info is [here](https://docs.aws.amazon.com/forecast/latest/dg/metrics.html)

#### Predictor algorithms

In [5]:
prophet_algorithmArn = 'arn:aws:forecast:::algorithm/Prophet'
deepAR_Plus_algorithmArn = 'arn:aws:forecast:::algorithm/Deep_AR_Plus'

### Prophet

In [6]:
# Prophet Specifics
prophet_predictorName= project+'_prophet_algo_1' + target_suffix + suffix

In [7]:
# Build Prophet:
prophet_create_predictor_response=forecast.create_predictor(
      PredictorName=prophet_predictorName, 
      AlgorithmArn=prophet_algorithmArn,
      ForecastHorizon=forecastHorizon,
      PerformAutoML= False,
      PerformHPO=False,
      EvaluationParameters= {"NumberOfBacktestWindows": NumberOfBacktestWindows, 
                             "BackTestWindowOffset": BackTestWindowOffset}, 
      InputDataConfig= {"DatasetGroupArn": target_datasetGroupArn, 
                        "SupplementaryFeatures": [ 
                         { 
                            "Name": "holiday",
                            "Value": "US"
                         }
                      ]},
      FeaturizationConfig= {"ForecastFrequency": ForecastFrequency, 
                            "ForecastDimensions":
                            ["store"],
                            "Featurizations": 
                            [
                              {"AttributeName": "target_value", 
                               "FeaturizationPipeline": 
                                [
                                  {"FeaturizationMethodName": "filling", 
                                   "FeaturizationMethodParameters": 
                                    {"frontfill": "none", 
                                     "middlefill": "zero", 
                                     "backfill": "zero"}
                                  }
                                ]
                              }
                            ]
                           }
                                                 )




### DeepAR+

In [8]:
# DeepAR+ Specifics
deeparp_predictorName= project+'_deeparp_algo_1' + suffix

In [9]:
# Build DeepAR+:
deeparp_create_predictor_response=forecast.create_predictor(
      PredictorName=deeparp_predictorName, 
      AlgorithmArn=deepAR_Plus_algorithmArn,
      ForecastHorizon=forecastHorizon,
      PerformAutoML= False,
      PerformHPO=False,
      EvaluationParameters= {"NumberOfBacktestWindows": NumberOfBacktestWindows, 
                             "BackTestWindowOffset": BackTestWindowOffset}, 
      InputDataConfig= {"DatasetGroupArn": target_datasetGroupArn, "SupplementaryFeatures": [ 
                         { 
                            "Name": "holiday",
                            "Value": "US"
                         }
                      ]},
      FeaturizationConfig= {"ForecastFrequency": ForecastFrequency, 
                            "ForecastDimensions":
                            ["store"],
                            
                            "Featurizations": 
                            [
                              {"AttributeName": "target_value", 
                               "FeaturizationPipeline": 
                                [
                                  {"FeaturizationMethodName": "filling", 
                                   "FeaturizationMethodParameters": 
                                    {"frontfill": "none", 
                                     "middlefill": "zero", 
                                     "backfill": "zero"}
                                  }
                                ]
                              }
                            ]
                           }
                                                 )





## Check if the predictors are active


In [10]:
# forecast.list_predictors(Filters=[
#     {"Condition": "IS",
#      "Key": "DatasetGroupArn", 
#      "Value": datasetGroupArn}
# ])

In [11]:
#prophet_predictor_arn = 'arn:aws:forecast:us-east-2:057716757052:predictor/WalmartKaggleGonsoo_prophet_algo_174655'
target_prophet_predictorArn = prophet_create_predictor_response["PredictorArn"]

In [12]:
target_deepar_predictorArn = deeparp_create_predictor_response["PredictorArn"]

In [19]:
%%time
# Check the Prophet status

while True:
    createProphetPredictorStatus = forecast.describe_predictor(PredictorArn= target_prophet_predictorArn)['Status']
    createDeeparpPredictorStatus = forecast.describe_predictor(PredictorArn= target_deepar_predictorArn)['Status']    
    print("Prophet: ", createProphetPredictorStatus)
    print("DeepARP: ", createDeeparpPredictorStatus)          
    if createProphetPredictorStatus != 'ACTIVE' and createProphetPredictorStatus != 'CREATE_FAILED':
        sleep(60)
    elif createDeeparpPredictorStatus != 'ACTIVE' and createDeeparpPredictorStatus != 'CREATE_FAILED':
        sleep(60)
    else:
        break

Prophet:  ACTIVE
DeepARP:  ACTIVE
CPU times: user 3.69 ms, sys: 2.8 ms, total: 6.5 ms
Wall time: 71.5 ms


## Predictor evaluation

#### Prophet

In [20]:
# Prophet Metrics
# prophet_arn = prophet_create_predictor_response['PredictorArn']
prophet_metrics = forecast.get_accuracy_metrics(PredictorArn=target_prophet_predictorArn)
pp = pprint.PrettyPrinter()
pp.pprint(prophet_metrics)

{'PredictorEvaluationResults': [{'AlgorithmArn': 'arn:aws:forecast:::algorithm/Prophet',
                                 'TestWindows': [{'EvaluationType': 'SUMMARY',
                                                  'Metrics': {'RMSE': 8.458201855444848,
                                                              'WeightedQuantileLosses': [{'LossValue': 0.05082779522512541,
                                                                                          'Quantile': 0.9},
                                                                                         {'LossValue': 0.10918503424565745,
                                                                                          'Quantile': 0.5},
                                                                                         {'LossValue': 0.046653896144418935,
                                                                                          'Quantile': 0.1}]}},
                                           

#### DeepAR+

In [21]:
# DeepAR+ Metrics
# deepar_predictor_arn = deeparp_create_predictor_response['PredictorArn']
deeparp_metrics = forecast.get_accuracy_metrics(PredictorArn=target_deepar_predictorArn)
pp = pprint.PrettyPrinter()
pp.pprint(deeparp_metrics)

{'PredictorEvaluationResults': [{'AlgorithmArn': 'arn:aws:forecast:::algorithm/Deep_AR_Plus',
                                 'TestWindows': [{'EvaluationType': 'SUMMARY',
                                                  'Metrics': {'RMSE': 8.6278037788792,
                                                              'WeightedQuantileLosses': [{'LossValue': 0.05153288965983034,
                                                                                          'Quantile': 0.9},
                                                                                         {'LossValue': 0.11113228665599806,
                                                                                          'Quantile': 0.5},
                                                                                         {'LossValue': 0.04729165976453416,
                                                                                          'Quantile': 0.1}]}},
                                         

In [22]:
%store target_prophet_predictorArn
%store target_deepar_predictorArn

Stored 'target_prophet_predictorArn' (str)
Stored 'target_deepar_predictorArn' (str)
